In [ ]:
# emoji_sentiment_analysis\notebooks\inference_app.ipynb

import sys
from pathlib import Path

# Add the project's root directory to the Python path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Imports for the app
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from loguru import logger
from emoji_sentiment_analysis.config import MODELS_DIR

# --- Load Model and Vectorizer ---
try:
    model = joblib.load(MODELS_DIR / "sentiment_model.pkl")
    vectorizer = joblib.load(MODELS_DIR / "tfidf_vectorizer.pkl")
    logger.success("Sentiment analyzer loaded successfully! 🚀")
except FileNotFoundError:
    logger.error("Model files not found. Please run the training notebook first.")
    model, vectorizer = None, None

# --- Preprocessing Helpers ---
EMOJI_PATTERN = re.compile(
    r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+',
    flags=re.UNICODE
)

def extract_emojis(text: str) -> str:
    """Extracts all emojis from a string."""
    return " ".join(re.findall(EMOJI_PATTERN, str(text)))

def preprocess_text(text: str) -> str:
    """Combines original text with extracted emojis for vectorization."""
    text = (text or "").strip()
    emojis = extract_emojis(text)
    return f"{text} {emojis}".strip()

# --- Widgets ---
text_input = widgets.Textarea(
    description='Enter a sentence:',
    placeholder='I love sentiment analysis but it is too hard to study 😔',
    layout=widgets.Layout(width='100%', height='100px')
)
sentiment_button = widgets.Button(
    description='Say your Sentiments!',
    button_style='success'
)
output_area = widgets.Output()

display(text_input, sentiment_button, output_area)

# --- Callback Function ---
def on_button_click(b):
    with output_area:
        clear_output()
        
        if not model or not vectorizer:
            print("Model not loaded. Please ensure the training notebook was run.")
            return

        input_text = text_input.value
        if not input_text or not input_text.strip():
            print("Please enter some text.")
            return

        prepared_text = preprocess_text(input_text)
        text_vectorized = vectorizer.transform([prepared_text])
        
        prediction = int(model.predict(text_vectorized)[0])
        sentiment_labels = {0: "NEGATIVE", 1: "POSITIVE"}
        sentiment_label = sentiment_labels.get(prediction, "UNKNOWN")

        try:
            proba = float(model.predict_proba(text_vectorized)[0].max())
        except (AttributeError, IndexError):
            proba = None

        print("==========================================================")
        print(f'YOUR INPUT IS OF "{sentiment_label}" SENTIMENT')
        if proba is not None:
            print(f"Confidence: {proba:.2%}")
        print("----------------------------------------------------------")
        print(f'Your input was: "{input_text.strip()}"')
        print("==========================================================")

# --- Register Handler ---
sentiment_button.on_click(on_button_click)
logger.info("Widgets are ready. Type a sentence and click the button.")